In [1]:
import csv
import pandas as pd
import numpy as np
from tensorflow import keras

In [2]:
df = pd.read_csv('C:/git_project/natural/data/temp_dataset/X_train.csv')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,428,20,326,611
1,0,0,0,0,0,0,0,0,0,0,...,810,419,28,558,1,77,1199,25,677,23
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,10,3,202,2
3,0,0,0,0,0,0,0,0,0,0,...,0,6005,102,5792,199,51,6,7,32,3422
4,0,0,0,0,0,0,0,0,0,0,...,1,2501,1015,213,2,10001,12019,1012,1267,213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145077,0,0,0,0,0,0,0,0,0,0,...,0,0,0,195,414,82,673,6,267,7031
145078,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,32,10,196
145079,0,0,0,0,0,0,0,0,0,0,...,36,1351,8,5698,13781,35,9899,9843,8,1045
145080,0,0,0,0,0,0,0,0,0,0,...,0,0,0,955,1,2171,2788,1145,379,7503


In [2]:
X_train_data = pd.read_csv('C:/git_project/natural/data/temp_dataset/X_train.csv')   # 데이터 불러오기
X_test_data = pd.read_csv('C:/git_project/natural/data/temp_dataset/X_test.csv')
y_train_data = pd.read_csv('C:/git_project/natural/data/temp_dataset/y_train.csv')   # 데이터 불러오기
y_test_data = pd.read_csv('C:/git_project/natural/data/temp_dataset/y_test.csv')

In [3]:
X_train_data.dropna(how='any', inplace=True)
X_test_data.dropna(how='any', inplace=True)
y_train_data.dropna(how='any', inplace=True)
y_test_data.dropna(how='any', inplace=True)

In [4]:
train_X = np.array(X_train_data)
test_X = np.array(X_test_data)
train_X, test_X

(array([[   0,    0,    0, ...,   20,  326,  611],
        [   0,    0,    0, ...,   25,  677,   23],
        [   0,    0,    0, ...,    3,  202,    2],
        ...,
        [   0,    0,    0, ..., 9843,    8, 1045],
        [   0,    0,    0, ..., 1145,  379, 7503],
        [   0,    0,    0, ...,   53, 1274,    1]], dtype=int64),
 array([[    0,     0,     0, ...,     0,   701,    84],
        [    0,     0,     0, ...,    47,   693,    22],
        [    0,     0,     0, ...,    62,   136,   244],
        ...,
        [    0,     0,     0, ...,   144,  3162,    26],
        [    0,     0,     0, ...,   415,     5, 12539],
        [    0,     0,     0, ...,     0,   861,     6]], dtype=int64))

In [5]:
train_y = np.array(y_train_data)
test_y = np.array(y_test_data)
train_y, test_y

(array([[0],
        [1],
        [0],
        ...,
        [0],
        [1],
        [0]], dtype=int64),
 array([[1],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]], dtype=int64))

# Model Building

In [11]:
from tensorflow import keras

vocab_size = 18208

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16, mask_zero=True))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Bidirectional(keras.layers.GRU(64)))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(8, activation='tanh'))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 16)          291328    
                                                                 
 dropout_3 (Dropout)         (None, None, 16)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              31488     
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 8)                 1032      
                                                                 
 dropout_5 (Dropout)         (None, 8)                 0         
                                                      

In [12]:
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Training

In [13]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((X_train_data, y_train_data))
val_dataset = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_data))

train_dataset = train_dataset.shuffle(buffer_size=1024, reshuffle_each_iteration=True).batch(128)
val_dataset = val_dataset.batch(1000)

train_dataset = train_dataset.prefetch(4)

In [ ]:
history = model.fit(train_dataset, epochs=20, validation_data=val_dataset, verbose=1)

Epoch 1/20
1134/1134 [==============================] - 53s 42ms/step - loss: 0.4298 - accuracy: 0.8046 - val_loss: 0.3690 - val_accuracy: 0.8373
Epoch 2/20
 162/1134 [===>..........................] - ETA: 39s - loss: 0.3676 - accuracy: 0.8475

# Evaluation

In [ ]:
# model.predict([encode_text(tokenize("이거 잼있네"), 32)])[0]

In [34]:
# model.predict([encode_text(tokenize("이거 졸잼이네"), 32)])[0]

array([0.13185185], dtype=float32)

In [ ]:
# model.predict([encode_text(tokenize("이거 개꿀잼"), 32)])[0]

In [ ]:
# model.predict([encode_text(tokenize("이거 노잼이네"), 32)])[0]

In [33]:
# model.predict([encode_text(tokenize("완전 핵노잼"), 32)])[0]

array([0.01995486], dtype=float32)

In [32]:
# results = model.evaluate(test_x, test_y, verbose=2)

1563/1563 - 7s - loss: 0.4108 - accuracy: 0.8068 - 7s/epoch - 5ms/step


In [31]:
# history_dict = history.history
# history_dict.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

# EDA

In [ ]:
# import matplotlib.pyplot as plt

# acc = history_dict['accuracy']
# val_acc = history_dict['val_accuracy']
# loss = history_dict['loss']
# val_loss = history_dict['val_loss']

# epochs = range(1, len(acc) + 1)

# # "bo"는 "파란색 점"입니다
# plt.plot(epochs, loss, 'bo', label='Training loss')
# # b는 "파란 실선"입니다
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()

# plt.show()

In [ ]:
# plt.clf()   # 그림을 초기화합니다

# plt.plot(epochs, acc, 'bo', label='Training acc')
# plt.plot(epochs, val_acc, 'b', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.show()